In [1]:
import pandas as pd
import os
#import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
from tensorflow.contrib.layers import fully_connected, batch_norm
import math
import numpy as np
import random

In [2]:
df = pd.read_csv('./DL_final_project/DL_Taiwan_data/sinica/201701_Taiwan.csv',
                 usecols=['Date','Time','PM2.5','PM10','PM1','Temperature','Humidity','lon','lat'])
#把201701_Taiwan.csv的header的' lat',' lon'改成'lat','lon' (多了空格)
#len(df)

In [3]:
def getAreaSeriesFromIdx(s):
    #f = s.mask(s['in_area'], s['lon_g']*3+s['lat_g']+1, axis=0)['lat']
    f = s.mask(s['in_area'], s['lat_g']+1, axis=0)['lat']
    f = f.where(s['in_area'], 0)
    return f

In [10]:
#if train_for_taiwan:
#    df = df[(df['lat']>= 21 )& (df['lat'] <= 27)&(df['lon']>=118)&(df['lon']<=122)]


In [11]:
df

,Date,Time,PM2.5,PM10,PM1,Temperature,Humidity,lat,lon,Timestamp,Hour,in_area,lon_g,lat_g,area
0,2017-01-01,08:00:00,31.0,33.0,22.0,22.75,78.0,25.072,121.657,2017-01-01 08:00:00,8,True,1.0,2.0,3.0
1,2017-01-01,08:00:00,59.0,76.0,40.0,21.25,92.0,22.963,120.325,2017-01-01 08:00:00,8,True,1.0,0.0,1.0
2,2017-01-01,08:00:00,31.0,33.0,22.0,22.75,78.0,25.072,121.657,2017-01-01 08:00:00,8,True,1.0,2.0,3.0
3,2017-01-01,08:00:00,59.0,76.0,40.0,21.25,92.0,22.963,120.325,2017-01-01 08:00:00,8,True,1.0,0.0,1.0
4,2017-01-01,08:00:00,31.0,33.0,22.0,22.75,78.0,25.072,121.657,2017-01-01 08:00:00,8,True,1.0,2.0,3.0
5,2017-01-01,08:00:00,59.0,76.0,40.0,21.25,92.0,22.963,120.325,2017-01-01 08:00:00,8,True,1.0,0.0,1.0
6,2017-01-01,08:00:01,39.0,46.0,28.0,23.62,77.0,24.167,120.692,2017-01-01 08:00:01,8,True,1.0,1.0,2.0
7,2017-01-01,08:00:01,39.0,46.0,28.0,23.62,77.0,24.167,120.692,2017-01-01 08:00:01,8,True,1.0,1.0,2.0
8,2017-01-01,08:00:01,39.0,46.0,28.0,23.62,77.0,24.167,120.692,2017-01-01 08:00:01,8,True,1.0,1.0,2.0
9,2017-01-01,08:00:01,70.0,87.0,52.0,23.50,85.0,22.853,120.546,2017-01-01 08:00:01,8,True,1.0,0.0,1.0


In [4]:
root_logdir = "tf_logs"
batch_log_step = 50
early_stopping_epochs = 50

In [5]:
def dnn(area_num, export_dir, X_1, y_1, X_2, y_2):
    tf.reset_default_graph()
    
    print("model will be exported in",export_dir)
    
    # logs
    start_time = datetime.now()
    now = start_time.strftime("%Y%m%d_%H%M%S")
    logdir = "./{}/run-{}".format(root_logdir, now)
    print("now=",now)
    
    # dnn graph defs
    n_input = 7
    n_epochs = 500
    n_hidden = [100,100,10]
    act_fn = tf.nn.sigmoid
    learning_rate = 0.005
    batch_normalization = False
    batch_size = 5000
    #batch_size = 1000 #小範圍測試用
    mult_bias = 1000

    # I/O
    with tf.name_scope("Input"):
        X = tf.placeholder(tf.float32, [None, n_input], name="X")
        is_training = tf.placeholder(tf.bool, shape=(), name="is_training")
    with tf.name_scope("Output"):
        y = tf.placeholder(tf.float32, [None, 1], name="y")
        y_biased = y/mult_bias
    
    # batch norm 
    he_init = tf.contrib.layers.variance_scaling_initializer()
    with tf.name_scope("BatchNormArgs"):
        bn_params = {
            'is_training': is_training,
            'decay': 0.99,
            'updates_collections': None,
            'scale': True
        }
    
    # DNN
    with tf.name_scope("DNN"):
        with tf.contrib.framework.arg_scope(
                [fully_connected],
                weights_initializer = he_init,
                normalizer_fn = batch_norm if batch_normalization else None,
                normalizer_params = bn_params if batch_normalization else None
                ):
            h1=fully_connected(X ,n_hidden[0],activation_fn=act_fn,scope="h1")
            h2=fully_connected(h1,n_hidden[1],activation_fn=act_fn,scope="h2")
            h3=fully_connected(h2,n_hidden[2],activation_fn=act_fn,scope="h3")
        logits=fully_connected(h3, 1, weights_initializer=he_init, activation_fn=act_fn,scope="out")
    
    with tf.name_scope("Cost"):
        #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y), name="cost")
        cost = tf.losses.mean_squared_error(logits, y_biased)
    with tf.name_scope("AdamOptimizer"):
        optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
        minimizer = optimizer.minimize(cost)
    
    init = tf.global_variables_initializer()
    with tf.name_scope("ModelSaver"):
        saver = tf.train.Saver()
    
    with tf.name_scope("Predict"):
        predict = tf.multiply(logits, mult_bias, name="predict")
        
    with tf.name_scope("Error"):
        ave_of_batch_y = tf.reduce_mean(y_biased)
        error = tf.abs(logits - y_biased)/ave_of_batch_y
        relative_err = tf.reduce_mean(error)
    
    with tf.name_scope("Summaries-Train"):
        cost_summary = tf.summary.scalar('cost_function',cost)
        error_summary = tf.summary.scalar('relative_err',relative_err)
    with tf.name_scope("Summaries-Validation"):
        v_cost_summary = tf.summary.scalar('v_cost_function',cost)
        v_error_summary = tf.summary.scalar('v_relative_err',relative_err)
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
    
    with tf.Session() as sess:
        sess.run(init)
        
        total_batch = len(X_1)//batch_size
        #total_batch = validation_idx_start//batch_size
        print("Total batch:",total_batch)
        #X_va, y_va = dfa_X[validation_idx_start:test_idx_start], dfa_Y[validation_idx_start:test_idx_start]
        X_va = X_2
        y_va = y_2
        #X_test = X_3
        #y_test = y_3
        
        best_va_err_triggered = False
        best_va_err = 0
        early_stopping_triggered = False
        stopping_epoch = 0
        step = 0
        save_path = ""
        best_save_path = ""
        
        perm = np.arange(len(X_1))
        np.random.shuffle(perm)
        X_t = X_1[perm]
        y_t = y_1[perm]
        
        for epoch in range(n_epochs):
            for batch in range(total_batch):
                X_ba = X_t[batch*batch_size: (batch+1)*batch_size]
                y_ba = y_t[batch*batch_size: (batch+1)*batch_size]
                _, c = sess.run([minimizer, cost], feed_dict = {is_training: True, X:X_ba, y:y_ba})
                
                if batch % batch_log_step == 0:
                    print("Epoch %4d batch %5d"%(epoch,batch))
                    cost_summary_str = cost_summary.eval(feed_dict={is_training:False, X:X_ba, y:y_ba})
                    step = epoch * total_batch + batch
                    file_writer.add_summary(cost_summary_str, step)
                    error_summary_str = error_summary.eval(feed_dict={is_training:False, X:X_ba, y:y_ba})
                    file_writer.add_summary(error_summary_str, step)
            
            save_path = saver.save(sess, "./checkpoint/model_"+now+".ckpt")
            va_err, va_cost = sess.run([relative_err, cost], feed_dict={is_training:False, X:X_va, y:y_va})
            va_l = sess.run(predict, feed_dict={is_training:False, X:X_va, y:y_va})
            rnd_result_idx = random.randint(0, len(X_va)-1)
            
            print(va_l[rnd_result_idx], y_va[rnd_result_idx]) # print 1st prediction result
            v_error_summary_str = v_error_summary.eval(feed_dict={is_training:False, X:X_va, y:y_va})
            file_writer.add_summary(v_error_summary_str, step)
            v_cost_summary_str = v_cost_summary.eval(feed_dict={is_training:False, X:X_va, y:y_va})
            file_writer.add_summary(v_cost_summary_str, step)
            print("Epoch %4d val.cost %3.6f val.err %3.2f%%"%(epoch,va_cost,va_err*100),end=" ")
            
            if best_va_err_triggered:
                if va_err < best_va_err:
                    print("best")
                    stopping_epoch = 0
                    best_va_err = va_err
                    
                    best_save_path = saver.save(sess, "./best_model/model_"+now)
                else:
                    stopping_epoch += 1
                    print("stopping %3d"%stopping_epoch)
                if stopping_epoch >= early_stopping_epochs:
                    early_stopping_triggered = True
                    print("Early stopping triggered: Step: %10d, val.err %3.2f%%"%(step, va_err*100))
            else:
                best_va_err = va_err
                best_va_err_triggered = True
                print("best")
                best_save_path = saver.save(sess, "./best_model/model_"+now)
            
            if early_stopping_triggered:
                break
        
        finish_time = datetime.now()
        print("best model saved to:", best_save_path)
        file_writer.close()
        elapse_time = finish_time - start_time
        total_seconds = elapse_time.total_seconds()
        print("Total time:", total_seconds)
        
        #X_test = dfa_X[test_idx_start:idx_end]
        #y_test = dfa_Y[test_idx_start:idx_end]
        saver.restore(sess, "./best_model/model_"+now)
        #best_err = relative_err.eval({is_training: False, X: X_test, y: y_test})
        #print("Test Err: %3.2f%%"%(best_err*100))
        
        builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
        builder.add_meta_graph_and_variables(sess, ["tag"], signature_def_map= {
            "model": tf.saved_model.signature_def_utils.predict_signature_def(
                inputs= {"X": X, "is_training": is_training},
                outputs= {"predict": predict})
        })
        builder.save()
        
        """
        print("Predicting...")
        predict_values = sess.run(predict, feed_dict={is_training: False, X: X_w})
        print("Predict End")
        return predict_values
        """  

In [6]:
# df2: validation
# df3: test
df_v = pd.read_csv('./DL_final_project/DL_Taiwan_data/sinica/201702_Taiwan.csv',
                 usecols=['Date','Time','PM2.5','PM10','PM1','Temperature','Humidity','lon','lat'])
test_idx_start = math.floor(len(df_v)*0.5)
df2 = df_v[:test_idx_start]
df3 = df_v[test_idx_start:]


In [7]:
df = df.assign(Timestamp = pd.to_datetime(df['Date']+' '+df['Time']))
df = df.assign(Hour = lambda x: x['Timestamp'].dt.hour)
df = df.assign(in_area = lambda x:(x['lat']>= 21 )& (x['lat'] <= 27)&(x['lon']>=118)&(x['lon']<=122))
df = df.assign(lon_g = lambda x:(x.lon-118)//2)
df = df.assign(lat_g = lambda x:(x.lat-21)//2)
df = df.assign(area = getAreaSeriesFromIdx)

df2 = df2.assign(Timestamp = pd.to_datetime(df2['Date']+' '+df2['Time']))
df2 = df2.assign(Hour = lambda x: x['Timestamp'].dt.hour)
df2 = df2.assign(in_area = lambda x:(x['lat']>= 21 )& (x['lat'] <= 27)&(x['lon']>=118)&(x['lon']<=122))
df2 = df2.assign(lon_g = lambda x:(x.lon-118)//2)
df2 = df2.assign(lat_g = lambda x:(x.lat-21)//2)
df2 = df2.assign(area = getAreaSeriesFromIdx)

In [12]:
df

,Date,Time,PM2.5,PM10,PM1,Temperature,Humidity,lat,lon,Timestamp,Hour,in_area,lon_g,lat_g,area
0,2017-01-01,08:00:00,31.0,33.0,22.0,22.75,78.0,25.072,121.657,2017-01-01 08:00:00,8,True,1.0,2.0,3.0
1,2017-01-01,08:00:00,59.0,76.0,40.0,21.25,92.0,22.963,120.325,2017-01-01 08:00:00,8,True,1.0,0.0,1.0
2,2017-01-01,08:00:00,31.0,33.0,22.0,22.75,78.0,25.072,121.657,2017-01-01 08:00:00,8,True,1.0,2.0,3.0
3,2017-01-01,08:00:00,59.0,76.0,40.0,21.25,92.0,22.963,120.325,2017-01-01 08:00:00,8,True,1.0,0.0,1.0
4,2017-01-01,08:00:00,31.0,33.0,22.0,22.75,78.0,25.072,121.657,2017-01-01 08:00:00,8,True,1.0,2.0,3.0
5,2017-01-01,08:00:00,59.0,76.0,40.0,21.25,92.0,22.963,120.325,2017-01-01 08:00:00,8,True,1.0,0.0,1.0
6,2017-01-01,08:00:01,39.0,46.0,28.0,23.62,77.0,24.167,120.692,2017-01-01 08:00:01,8,True,1.0,1.0,2.0
7,2017-01-01,08:00:01,39.0,46.0,28.0,23.62,77.0,24.167,120.692,2017-01-01 08:00:01,8,True,1.0,1.0,2.0
8,2017-01-01,08:00:01,39.0,46.0,28.0,23.62,77.0,24.167,120.692,2017-01-01 08:00:01,8,True,1.0,1.0,2.0
9,2017-01-01,08:00:01,70.0,87.0,52.0,23.50,85.0,22.853,120.546,2017-01-01 08:00:01,8,True,1.0,0.0,1.0


In [8]:
area_num = 4
# check folder
export_check_num = 1
export_dir = './final_model/'
while os.path.exists(export_dir):
    export_check_num += 1
    export_dir = './final_model_'+str(export_check_num)+'/'
for a in range(area_num):
    print('area', a)
    
    dfa = df[df['area'] == a]
    df_X = dfa[['Hour','PM10','PM1','Temperature','Humidity','lon','lat']]
    df_X = df_X.values
    df_Y = dfa[['PM2.5']]
    df_Y = df_Y.values
    print('train len =', len(df_X))
    
    df2a = df2[df2['area']==a]
    df2_X = df2a[['Hour','PM10','PM1','Temperature','Humidity','lon','lat']]
    df2_X = df2_X.values
    df2_Y = df2a[['PM2.5']]
    df2_Y = df2_Y.values
    print('test len =',len(df2_X))
    
    
    export_dir_area = export_dir + 'area_'+str(a)+"/"
    dfa.describe(include="all")
    dnn(a, export_dir_area, df_X, df_Y, df2_X, df2_Y)

area 0
train len = 339197
test len = 97343
model will be exported in ./final_model_9/area_0/
now= 20180617_113432
Total batch: 67
Epoch    0 batch     0
Epoch    0 batch    50
[ 126.99173737] [19]
Epoch    0 val.cost 0.010656 val.err 408.08% best
Epoch    1 batch     0
Epoch    1 batch    50
[ 89.06938171] [18]
Epoch    1 val.cost 0.004361 val.err 256.87% best
Epoch    2 batch     0
Epoch    2 batch    50
[ 70.58782959] [43]
Epoch    2 val.cost 0.002334 val.err 183.65% best
Epoch    3 batch     0
Epoch    3 batch    50
[ 59.70393753] [18]
Epoch    3 val.cost 0.001459 val.err 141.63% best
Epoch    4 batch     0
Epoch    4 batch    50
[ 52.54442215] [42]
Epoch    4 val.cost 0.001014 val.err 115.27% best
Epoch    5 batch     0
Epoch    5 batch    50
[ 47.48961258] [46]
Epoch    5 val.cost 0.000761 val.err 98.00% best
Epoch    6 batch     0
Epoch    6 batch    50
[ 43.73590851] [10]
Epoch    6 val.cost 0.000606 val.err 86.32% best
Epoch    7 batch     0
Epoch    7 batch    50
[ 40.85722733

Epoch   70 batch    50
[ 34.93763351] [36]
Epoch   70 val.cost 0.000040 val.err 11.70% best
Epoch   71 batch     0
Epoch   71 batch    50
[ 14.97287369] [16]
Epoch   71 val.cost 0.000040 val.err 11.65% best
Epoch   72 batch     0
Epoch   72 batch    50
[ 24.37457275] [26]
Epoch   72 val.cost 0.000040 val.err 11.61% best
Epoch   73 batch     0
Epoch   73 batch    50
[ 38.60558701] [37]
Epoch   73 val.cost 0.000040 val.err 11.57% best
Epoch   74 batch     0
Epoch   74 batch    50
[ 13.16304016] [6]
Epoch   74 val.cost 0.000040 val.err 11.52% best
Epoch   75 batch     0
Epoch   75 batch    50
[ 13.12198639] [6]
Epoch   75 val.cost 0.000040 val.err 11.44% best
Epoch   76 batch     0
Epoch   76 batch    50
[ 14.74321461] [15]
Epoch   76 val.cost 0.000040 val.err 11.48% stopping   1
Epoch   77 batch     0
Epoch   77 batch    50
[ 14.71901321] [13]
Epoch   77 val.cost 0.000040 val.err 11.43% best
Epoch   78 batch     0
Epoch   78 batch    50
[ 50.94132996] [51]
Epoch   78 val.cost 0.000040 va

Epoch  139 val.cost 0.000022 val.err 8.48% stopping  15
Epoch  140 batch     0
Epoch  140 batch    50
[ 13.51496696] [14]
Epoch  140 val.cost 0.000021 val.err 7.69% stopping  16
Epoch  141 batch     0
Epoch  141 batch    50
[ 13.69741154] [14]
Epoch  141 val.cost 0.000022 val.err 8.54% stopping  17
Epoch  142 batch     0
Epoch  142 batch    50
[ 12.04910755] [12]
Epoch  142 val.cost 0.000022 val.err 8.12% stopping  18
Epoch  143 batch     0
Epoch  143 batch    50
[ 29.69240189] [32]
Epoch  143 val.cost 0.000021 val.err 7.77% stopping  19
Epoch  144 batch     0
Epoch  144 batch    50
[ 22.53731537] [19]
Epoch  144 val.cost 0.000022 val.err 7.76% stopping  20
Epoch  145 batch     0
Epoch  145 batch    50
[ 7.07779169] [7]
Epoch  145 val.cost 0.000022 val.err 7.95% stopping  21
Epoch  146 batch     0
Epoch  146 batch    50
[ 5.65110779] [5]
Epoch  146 val.cost 0.000022 val.err 7.75% stopping  22
Epoch  147 batch     0
Epoch  147 batch    50
[ 18.98922539] [19]
Epoch  147 val.cost 0.000021

Epoch  207 val.cost 0.000021 val.err 6.80% stopping   1
Epoch  208 batch     0
Epoch  208 batch    50
[ 43.61744308] [45]
Epoch  208 val.cost 0.000021 val.err 6.88% stopping   2
Epoch  209 batch     0
Epoch  209 batch    50
[ 37.62459564] [37]
Epoch  209 val.cost 0.000021 val.err 6.70% best
Epoch  210 batch     0
Epoch  210 batch    50
[ 16.67046356] [17]
Epoch  210 val.cost 0.000022 val.err 6.93% stopping   1
Epoch  211 batch     0
Epoch  211 batch    50
[ 55.66426468] [58]
Epoch  211 val.cost 0.000021 val.err 6.81% stopping   2
Epoch  212 batch     0
Epoch  212 batch    50
[ 8.077981] [8]
Epoch  212 val.cost 0.000021 val.err 6.81% stopping   3
Epoch  213 batch     0
Epoch  213 batch    50
[ 10.5528841] [11]
Epoch  213 val.cost 0.000021 val.err 6.70% best
Epoch  214 batch     0
Epoch  214 batch    50
[ 22.65955162] [23]
Epoch  214 val.cost 0.000022 val.err 7.06% stopping   1
Epoch  215 batch     0
Epoch  215 batch    50
[ 25.35776329] [25]
Epoch  215 val.cost 0.000022 val.err 6.99% st

[ 59.06565475] [57]
Epoch  275 val.cost 0.000021 val.err 6.91% stopping  32
Epoch  276 batch     0
Epoch  276 batch    50
[ 6.02379322] [6]
Epoch  276 val.cost 0.000022 val.err 7.32% stopping  33
Epoch  277 batch     0
Epoch  277 batch    50
[ 15.90701199] [17]
Epoch  277 val.cost 0.000021 val.err 6.76% stopping  34
Epoch  278 batch     0
Epoch  278 batch    50
[ 29.071558] [30]
Epoch  278 val.cost 0.000021 val.err 6.89% stopping  35
Epoch  279 batch     0
Epoch  279 batch    50
[ 19.55906296] [19]
Epoch  279 val.cost 0.000021 val.err 6.81% stopping  36
Epoch  280 batch     0
Epoch  280 batch    50
[ 23.69937134] [24]
Epoch  280 val.cost 0.000021 val.err 6.96% stopping  37
Epoch  281 batch     0
Epoch  281 batch    50
[ 31.32006836] [31]
Epoch  281 val.cost 0.000021 val.err 6.82% stopping  38
Epoch  282 batch     0
Epoch  282 batch    50
[ 8.57378006] [10]
Epoch  282 val.cost 0.000021 val.err 7.16% stopping  39
Epoch  283 batch     0
Epoch  283 batch    50
[ 9.68435574] [10]
Epoch  283

...

Epoch   84 batch    50
Epoch   84 batch   100
Epoch   84 batch   150
Epoch   84 batch   200
Epoch   84 batch   250
Epoch   84 batch   300
Epoch   84 batch   350
Epoch   84 batch   400
Epoch   84 batch   450
[ 22.89248085] [23]
Epoch   84 val.cost 0.000060 val.err 6.93% stopping  16
Epoch   85 batch     0
Epoch   85 batch    50
Epoch   85 batch   100
Epoch   85 batch   150
Epoch   85 batch   200
Epoch   85 batch   250
Epoch   85 batch   300
Epoch   85 batch   350
Epoch   85 batch   400
Epoch   85 batch   450
[ 41.97088623] [42]
Epoch   85 val.cost 0.000061 val.err 7.28% stopping  17
Epoch   86 batch     0
Epoch   86 batch    50
Epoch   86 batch   100
Epoch   86 batch   150
Epoch   86 batch   200
Epoch   86 batch   250
Epoch   86 batch   300
Epoch   86 batch   350
Epoch   86 batch   400
Epoch   86 batch   450
[ 35.10997772] [35]
Epoch   86 val.cost 0.000059 val.err 6.86% stopping  18
Epoch   87 batch     0
Epoch   87 batch    50
Epoch   87 batch   100
Epoch   87 batch   150
Epoch   87 ba

Epoch  111 batch    50
Epoch  111 batch   100
Epoch  111 batch   150
Epoch  111 batch   200
Epoch  111 batch   250
Epoch  111 batch   300
Epoch  111 batch   350
Epoch  111 batch   400
Epoch  111 batch   450
[ 53.27297211] [52]
Epoch  111 val.cost 0.000087 val.err 7.12% stopping  43
Epoch  112 batch     0
Epoch  112 batch    50
Epoch  112 batch   100
Epoch  112 batch   150
Epoch  112 batch   200
Epoch  112 batch   250
Epoch  112 batch   300
Epoch  112 batch   350
Epoch  112 batch   400
Epoch  112 batch   450
[ 40.11835861] [40]
Epoch  112 val.cost 0.000087 val.err 7.12% stopping  44
Epoch  113 batch     0
Epoch  113 batch    50
Epoch  113 batch   100
Epoch  113 batch   150
Epoch  113 batch   200
Epoch  113 batch   250
Epoch  113 batch   300
Epoch  113 batch   350
Epoch  113 batch   400
Epoch  113 batch   450
[ 36.11226273] [34]
Epoch  113 val.cost 0.000097 val.err 7.30% stopping  45
Epoch  114 batch     0
Epoch  114 batch    50
Epoch  114 batch   100
Epoch  114 batch   150
Epoch  114 ba

Epoch   28 batch   100
Epoch   28 batch   150
Epoch   28 batch   200
[ 24.85319901] [61]
Epoch   28 val.cost 0.000612 val.err 37.97% best
Epoch   29 batch     0
Epoch   29 batch    50
Epoch   29 batch   100
Epoch   29 batch   150
Epoch   29 batch   200
[ 16.47974968] [6]
Epoch   29 val.cost 0.000600 val.err 36.87% best
Epoch   30 batch     0
Epoch   30 batch    50
Epoch   30 batch   100
Epoch   30 batch   150
Epoch   30 batch   200
[ 28.3419838] [45]
Epoch   30 val.cost 0.000595 val.err 36.45% best
Epoch   31 batch     0
Epoch   31 batch    50
Epoch   31 batch   100
Epoch   31 batch   150
Epoch   31 batch   200
[ 30.71051788] [22]
Epoch   31 val.cost 0.000592 val.err 36.10% best
Epoch   32 batch     0
Epoch   32 batch    50
Epoch   32 batch   100
Epoch   32 batch   150
Epoch   32 batch   200
[ 19.31568527] [20]
Epoch   32 val.cost 0.000591 val.err 36.16% stopping   1
Epoch   33 batch     0
Epoch   33 batch    50
Epoch   33 batch   100
Epoch   33 batch   150
Epoch   33 batch   200
[ 29.

Epoch   71 batch   200
[ 40.44218445] [39]
Epoch   71 val.cost 0.000585 val.err 35.41% stopping  12
Epoch   72 batch     0
Epoch   72 batch    50
Epoch   72 batch   100
Epoch   72 batch   150
Epoch   72 batch   200
[ 25.17166901] [25]
Epoch   72 val.cost 0.000585 val.err 35.73% stopping  13
Epoch   73 batch     0
Epoch   73 batch    50
Epoch   73 batch   100
Epoch   73 batch   150
Epoch   73 batch   200
[ 24.83452034] [44]
Epoch   73 val.cost 0.000584 val.err 35.30% best
Epoch   74 batch     0
Epoch   74 batch    50
Epoch   74 batch   100
Epoch   74 batch   150
Epoch   74 batch   200
[ 49.35253525] [47]
Epoch   74 val.cost 0.000584 val.err 35.19% best
Epoch   75 batch     0
Epoch   75 batch    50
Epoch   75 batch   100
Epoch   75 batch   150
Epoch   75 batch   200
[ 21.92539787] [15]
Epoch   75 val.cost 0.000583 val.err 35.30% stopping   1
Epoch   76 batch     0
Epoch   76 batch    50
Epoch   76 batch   100
Epoch   76 batch   150
Epoch   76 batch   200
[ 22.72760391] [0]
Epoch   76 val

Epoch  114 batch   200
[ 25.501194] [10]
Epoch  114 val.cost 0.000585 val.err 35.18% stopping  16
Epoch  115 batch     0
Epoch  115 batch    50
Epoch  115 batch   100
Epoch  115 batch   150
Epoch  115 batch   200
[ 34.38393402] [29]
Epoch  115 val.cost 0.000584 val.err 35.44% stopping  17
Epoch  116 batch     0
Epoch  116 batch    50
Epoch  116 batch   100
Epoch  116 batch   150
Epoch  116 batch   200
[ 20.62285042] [14]
Epoch  116 val.cost 0.000584 val.err 35.06% stopping  18
Epoch  117 batch     0
Epoch  117 batch    50
Epoch  117 batch   100
Epoch  117 batch   150
Epoch  117 batch   200
[ 100.17435455] [100]
Epoch  117 val.cost 0.000585 val.err 35.11% stopping  19
Epoch  118 batch     0
Epoch  118 batch    50
Epoch  118 batch   100
Epoch  118 batch   150
Epoch  118 batch   200
[ 30.11772919] [41]
Epoch  118 val.cost 0.000585 val.err 35.34% stopping  20
Epoch  119 batch     0
Epoch  119 batch    50
Epoch  119 batch   100
Epoch  119 batch   150
Epoch  119 batch   200
[ 25.55470276] [1

In [15]:
df3

,Date,Time,PM2.5,PM10,PM1,Temperature,Humidity,lat,lon
1384569,2017-02-13,04:05:37,56,74,40,18.62,92,22.608,120.271
1384570,2017-02-13,04:05:38,29,31,21,17.12,71,24.252,120.739
1384571,2017-02-13,04:05:38,49,62,35,20.50,81,22.514,120.395
1384572,2017-02-13,04:05:39,17,20,12,7.25,82,24.255,121.259
1384573,2017-02-13,04:05:40,37,43,26,19.12,65,22.993,120.213
1384574,2017-02-13,04:05:41,37,47,27,18.00,73,23.189,120.192
1384575,2017-02-13,04:05:41,53,70,38,20.12,71,22.622,120.286
1384576,2017-02-13,04:05:41,36,41,26,17.50,75,23.018,120.134
1384577,2017-02-13,04:05:42,54,71,38,18.62,82,22.708,120.438
1384578,2017-02-13,04:05:42,24,26,19,21.00,51,24.175,120.716


In [18]:
df3 = df3.assign(Timestamp = pd.to_datetime(df3['Date']+' '+df3['Time']))
df3 = df3.assign(Hour = lambda x: x['Timestamp'].dt.hour)
df3 = df3.assign(in_area = lambda x:(x['lat']>= 21 )& (x['lat'] <= 27)&(x['lon']>=118)&(x['lon']<=122))
df3 = df3.assign(lon_g = lambda x:(x.lon-118)//2)
df3 = df3.assign(lat_g = lambda x:(x.lat-21)//2)
df3 = df3.assign(area = getAreaSeriesFromIdx)
df3 = df3.reset_index(drop=True)

In [27]:
#[i for i in range (len(df3))]
df3 = df3.assign(idx = df3.index)
df3

,Date,Time,PM2.5,PM10,PM1,Temperature,Humidity,lat,lon,Timestamp,Hour,in_area,lon_g,lat_g,area,idx
0,2017-02-13,04:05:37,56,74,40,18.62,92,22.608,120.271,2017-02-13 04:05:37,4,True,1.0,0.0,1.0,0
1,2017-02-13,04:05:38,29,31,21,17.12,71,24.252,120.739,2017-02-13 04:05:38,4,True,1.0,1.0,2.0,1
2,2017-02-13,04:05:38,49,62,35,20.50,81,22.514,120.395,2017-02-13 04:05:38,4,True,1.0,0.0,1.0,2
3,2017-02-13,04:05:39,17,20,12,7.25,82,24.255,121.259,2017-02-13 04:05:39,4,True,1.0,1.0,2.0,3
4,2017-02-13,04:05:40,37,43,26,19.12,65,22.993,120.213,2017-02-13 04:05:40,4,True,1.0,0.0,1.0,4
5,2017-02-13,04:05:41,37,47,27,18.00,73,23.189,120.192,2017-02-13 04:05:41,4,True,1.0,1.0,2.0,5
6,2017-02-13,04:05:41,53,70,38,20.12,71,22.622,120.286,2017-02-13 04:05:41,4,True,1.0,0.0,1.0,6
7,2017-02-13,04:05:41,36,41,26,17.50,75,23.018,120.134,2017-02-13 04:05:41,4,True,1.0,1.0,2.0,7
8,2017-02-13,04:05:42,54,71,38,18.62,82,22.708,120.438,2017-02-13 04:05:42,4,True,1.0,0.0,1.0,8
9,2017-02-13,04:05:42,24,26,19,21.00,51,24.175,120.716,2017-02-13 04:05:42,4,True,1.0,1.0,2.0,9


In [41]:
round_to_int = False
result_pd = pd.DataFrame({'PM2.5':[],'idx':[]})
for a in range(area_num):
#if True:
    #a = 0
    print('Area', a)
    tf.reset_default_graph()
    dft = df3[df3['area']==a]
    predict_values = np.zeros(len(dft))
    print(len(dft))
    df_X = dft[['Hour','PM10','PM1','Temperature','Humidity','lon','lat']]
    df_X = df_X.values
    df_Y = dft[['PM2.5']]
    df_Y = df_Y.values
    
    batch_size = 10000
    disp_log = 10
    n_batches = math.ceil(len(dft) / batch_size)
    with tf.Session(graph=tf.Graph()) as sess:
        tf.saved_model.loader.load(sess, ["tag"], export_dir + 'area_'+str(a)+"/")
        graph = tf.get_default_graph()
        X = graph.get_tensor_by_name("Input/X:0")
        is_training = graph.get_tensor_by_name("Input/is_training:0")
        predict = graph.get_tensor_by_name("Predict/predict:0")

        for i in range(n_batches):
            if i % disp_log == 0:
                print('batch',i)
            if i == n_batches - 1:
                X_batch = df_X[i*batch_size:]
                predict_batch = sess.run(predict, {is_training: False, X: X_batch})
                predict_values[i*batch_size:] = predict_batch[:,0]
            else:
                X_batch = df_X[i*batch_size:(i+1)*batch_size]
                predict_batch = sess.run(predict, {is_training: False, X: X_batch})
                predict_values[i*batch_size:(i+1)*batch_size] = predict_batch[:,0]


        if round_to_int:
            predict_values = [int(v) for v in predict_values]
    
    area_result = pd.DataFrame({'PM2.5':predict_values, 'idx':dft['idx']})
    result_pd = result_pd.append(area_result)
print('end')

Area 0
117992
INFO:tensorflow:Restoring parameters from b'./final_model_5/area_0/variables\\variables'
batch 0
batch 10
Area 1
244601
INFO:tensorflow:Restoring parameters from b'./final_model_5/area_1/variables\\variables'
batch 0
batch 10
batch 20
Area 2
723014
INFO:tensorflow:Restoring parameters from b'./final_model_5/area_2/variables\\variables'
batch 0
batch 10
batch 20
batch 30
batch 40
batch 50
batch 60
batch 70
Area 3
298963
INFO:tensorflow:Restoring parameters from b'./final_model_5/area_3/variables\\variables'
batch 0
batch 10
batch 20
end


In [43]:
result_pd.sort_values('idx')

,PM2.5,idx
0,57.822571,0.0
1,27.380884,1.0
2,49.965599,2.0
3,15.638891,3.0
4,35.877102,4.0
5,38.026768,5.0
6,54.426346,6.0
7,35.131573,7.0
8,55.201263,8.0
9,23.718853,9.0
